# Zad. 

Otagować (czyli wskazać słowa "kluczowe", Latent Dirichlet Allocation) teksty korpusu 20newsgroups.

In [2]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import pandas as pd
import os 

### Można sczytać dane bez nagłówków i stopek

In [3]:
df = pd.read_csv('songdata.csv')
df['text'] = df['text'].str.replace('\n', '')
df.head(10)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face And i..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently ..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go Why I had to ...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong You're encha..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun Couldn't sleep...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby I'm sitting all alon...


In [30]:
y = df["artist"].values#[:1000]
X = df["text"].values#[:1000]

num=10
X_set = []
Y_set =[]
for i in np.unique(y):
    X_set = X_set+ list(X[ y == i ][:num])
    Y_set = Y_set + list(y[ y == i ][:num])

# Zad
Wykonaj przekształcenie Tf-Idf

```python
from sklearn.feature_extraction.text import TfidfVectorizer
```

In [37]:
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

def remove_punctuation(words):
    remove_list =  stopwords.words('english') + list(string.punctuation) + ["'s",'′','--', '."', '!"', '?"', ',"', '``', "''", "\\\\",'⁄',"n't","'ll","'m"]
    return [w for w in words if not w in set(remove_list)]

def remove_nummbers(words):
    return [w for w in words if not w.isdigit()]

def to_lower(words):
    return [w.lower() for w in words]


def my_tokenizer(sent):
    tokens = word_tokenize(sent)
    tokens = to_lower(tokens)    
    tokens = remove_punctuation(tokens)
    tokens = remove_nummbers(tokens)
    stemmer = PorterStemmer()
    #res = [stemmer.stem(word) for word in tokens]
    res = [word for word in tokens]
    return res 



tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, stop_words=stopwords.words('english') + list(string.punctuation))
tfidf_matrix = tfidf_vectorizer.fit_transform(X_set)
# print( tfidf_vectorizer.vocabulary_ )

In [38]:
print("----")
print(tfidf_matrix.todense().shape)
print(tfidf_matrix.todense())

----
(6304, 35701)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Zad 
Stwórz model języka w bibliotece **gensim** na podstawie reprezentacji **Tf-Idf**

In [39]:
corpus_sparse = matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)
dictionary_list = tfidf_vectorizer.get_feature_names()
dictionary_sparse_dict = {i:j for i,j in enumerate(dictionary_list)}
dictionary = corpora.Dictionary.from_corpus(corpus_sparse, id2word=dictionary_sparse_dict)

# Zad 
Stwórz model **Lda** (Latent Dirichlet Allocation) z
* **num_topics=20**

In [40]:
model = models.LdaModel(corpus=corpus_sparse, id2word=dictionary, num_topics=5)#, alpha='auto', eta='auto', iterations=100)

# Wizualizacja modelu LDA:

pyLDAvis

In [41]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [42]:
pyLDAvis.gensim.prepare(model, corpus_sparse, dictionary)

C:\Users\przem\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.142974 -0.005235       1        1  72.472824
3     -0.034733  0.014605       2        1  14.803018
0      0.067281 -0.003836       3        1   4.833935
2      0.040538 -0.013751       4        1   4.319731
4      0.069889  0.008218       5        1   3.570498, topic_info=      Category        Freq     Term       Total  loglift  logprob
term                                                             
21616  Default   90.000000    nigga   90.000000  30.0000  30.0000
13228  Default  115.000000      got  115.000000  29.0000  29.0000
12288  Default   24.000000     fuck   24.000000  28.0000  28.0000
28224  Default   43.000000     shit   43.000000  27.0000  27.0000
34956  Default   16.000000      wit   16.000000  26.0000  26.0000
35352  Default   67.000000     yeah   67.000000  25.0000  25.0000
3696   Default   27.000000    bitch   27.000000  24.0000  24.0000
22211  Default   75.000000       oh   75.000000  23.0000  23.0000
12805  Default   90.000000      get   90.000000  22.0000  22.0000
18198  Default  112.000000     like  112.000000  21.0000  21.0000
14665  Default    8.000000     hoes    8.000000  20.0000  20.0000
285    Default   32.000000      ...   32.000000  19.0000  19.0000
34702  Default   16.000000    white   16.000000  18.0000  18.0000
17824  Default   12.000000      lay   12.000000  17.0000  17.0000
1158   Default   82.000000       ai   82.000000  16.0000  16.0000
1638   Default   13.000000    angel   13.000000  15.0000  15.0000
18586  Default  137.000000     love  137.000000  14.0000  14.0000
8384   Default    5.000000      dem    5.000000  13.0000  13.0000
14800  Default    6.000000     hood    6.000000  12.0000  12.0000
3575   Default   25.000000      big   25.000000  11.0000  11.0000
20489  Default   34.000000    money   34.000000  10.0000  10.0000
7842   Default    6.000000       da    6.000000   9.0000   9.0000
3739   Default   30.000000    black   30.000000   8.0000   8.0000
21065  Default   99.000000       na   99.000000   7.0000   7.0000
35439  Default   16.000000       yo   16.000000   6.0000   6.0000
19167  Default   45.000000      man   45.000000   5.0000   5.0000
2572   Default   75.000000     baby   75.000000   4.0000   4.0000
14660  Default    5.000000      hoe    5.000000   3.0000   3.0000
10160  Default    9.000000       em    9.000000   2.0000   2.0000
17500  Default   13.000000       la   13.000000   1.0000   1.0000
...        ...         ...      ...         ...      ...      ...
25228   Topic5    0.129883   rachel    0.643119   1.7328  -9.2198
12360   Topic5    0.173232     funk    1.164875   1.4267  -8.9317
8384    Topic5    0.409883      dem    5.891653   0.6670  -8.0705
25319   Topic5    0.311033    rainy    4.306749   0.7044  -8.3465
6960    Topic5    0.158398  consume    1.042593   1.4481  -9.0213
17500   Topic5    0.502933       la   13.035888   0.0775  -7.8659
7267    Topic5    0.144748   counts    0.929086   1.4733  -9.1114
8121    Topic5    0.237743       de    3.389477   0.6752  -8.6152
9786    Topic5    0.167769    dusty    1.412563   1.2019  -8.9638
22211   Topic5    0.523059       oh   75.696793  -1.6423  -7.8267
18586   Topic5    0.554326     love  137.801529  -2.1834  -7.7686
199     Topic5    0.505143      're   99.966553  -1.9553  -7.8615
18198   Topic5    0.513716     like  112.466095  -2.0563  -7.8447
237     Topic5    0.426114      've   89.952278  -2.0199  -8.0317
17255   Topic5    0.453843     know  135.382690  -2.3656  -7.9686
13228   Topic5    0.432613      got  115.809250  -2.2574  -8.0165
27604   Topic5    0.408105      see  103.473854  -2.2031  -8.0749
12805   Topic5    0.388929      get   90.869835  -2.1213  -8.1230
13087   Topic5    0.385607       go   90.519737  -2.1260  -8.1316
5024    Topic5    0.378711       ca   93.471046  -2.1762  -8.1496
34406   Topic5    0.355056      way   69.275604  -1.9411  -8.2141
21065   Topic5 